## This notebook is for testing scraping. I'll be using the [Gwent:The Witcher Card Game website](https://www.playgwent.com/) to scrape the information I need.
## Tools: 
* Selenium
* BeautifulSoup
* Pandas
* Jupyter Notebook.

In [72]:
#selenium with chrome
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

driver = webdriver.Chrome('C:/Users/User/Downloads/chromedriver_win32/chromedriver.exe')

def get_stats(player):
    url = 'https://www.playgwent.com/es/profile/' + player
    driver.get(url)
    driver.implicitly_wait(5)
    #driver.refresh()
    driver.implicitly_wait(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    current_stats = soup.find('table', {'class': 'c-statistics-table current-ranked'})
    df = pd.read_html(str(current_stats))
    df[0].rename(columns = {'Clasificación de la temporada actual':'Stats', 'Clasificación de la temporada actual.1':'value'}, inplace = True)
    df=df[0].drop(['Clasificación de la temporada actual.2'], axis=1)
    df.value = df.value.str.replace('partidas', '')
    df = df.iloc[0:9,:]
    df1 = df.set_index('Stats').T
    df1['player'] = player
    return df1

C:\Users\User\AppData\Local\Temp/ipykernel_18348/1676004483.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/User/Downloads/chromedriver_win32/chromedriver.exe')


In [26]:
##Test with one name that I found
df1 = get_stats('GrandKazuzu')

In [101]:
url_better= 'https://masters.playgwent.com/en/rankings/masters-4/may-season'
## Obtain the best 20 players of the current season
def better(url):
    driver.get(url)
    driver.implicitly_wait(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    info = soup.find_all('div', {'class': 'c-ranking-table__tr'})
    #print(len(info))
    df_better = pd.DataFrame(columns = ['ranking', 'nick', 'matches', 'mmr'])
    for i in range(20):
        ranking = info[i].find('div', {'class': 'td-number'}).text
        nick = info[i].find('div', {'class': 'td-nick'}).text
        matches = info[i].find('div', {'class': 'td-matches'}).text
        mmr = info[i].find('div', {'class': 'td-mmr'}).text
        df_better.loc[i] = [ranking, nick, matches, mmr]
    return df_better


In [102]:
df_better = better(url_better)

40


In [103]:
df_better.head()

,ranking,nick,matches,mmr
0,1,GrandKazuzu,418 matches,"10,378\n"
1,2,Bart933,318 matches,"10,367\n"
2,3,John丶Sally,465 matches,"10,365\n"
3,4,infern-ale,469 matches,"10,242\n"
4,5,Nik_r,452 matches,"10,241\n"


In [104]:
#Obtain more stats by player (20 best players)
df_by_player = pd.DataFrame(columns = ['value','Total', 'Victorias', 'Derrotas', 'Draws', 'Reinos del Norte',
       'Sindicato', 'Monstruos', "Scoia'tael", 'Nilfgaard', 'player'] )
df_by_player.set_index('value', inplace=True)

In [105]:
for i in df_better.nick:
    df_by_player = pd.concat([df_by_player, get_stats(i)])

In [106]:
df_by_player.head()


,Total,Victorias,Derrotas,Draws,Reinos del Norte,Sindicato,Monstruos,Scoia'tael,Nilfgaard,player,Skellige
value,10.377 MMR415,271,140,4,84,85,113,132,1,GrandKazuzu,NaN
value,10.367 MMR318,221,93,4,77,107,0,75,59,Bart933,NaN
value,10.365 MMR465,303,154,8,NaN,82,73,123,92,John丶Sally,95
value,10.242 MMR467,281,174,12,68,NaN,164,76,102,infern-ale,40
value,10.241 MMR452,285,158,9,NaN,56,92,85,68,Nik_r,129
